In [ ]:
import os
import shutil

os.system("git clone https://github.com/matt-nann/AuthenticCursor.git")
try:
  shutil.copytree("AuthenticCursor/src", "src")
except:
  ...
shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
os.system("pip install -r requirementsGAN.txt")
shutil.rmtree("AuthenticCursor")

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
from src.mouseGAN.dataProcessing import MouseGAN_Data
from src.mouseGAN.dataset import getDataloader

USE_FAKE_DATA = True
dataset = MouseGAN_Data(USE_FAKE_DATA=USE_FAKE_DATA, equal_length=True, lowerLimit=50, upperLimit=80)
if USE_FAKE_DATA:
    dataset.loadFakeWindMouseData()
else:
    df_moves, df_trajectory = dataset.collectRawMouseTrajectories()

In [ ]:
norm_input_trajectories, norm_buttonTargets = dataset.processMouseData(SHOW_ALL=False, samples=5000)

## verifying the mean trajectory is centered around zero (even class distribution)

In [31]:
averageMove = np.array(dataset.input_trajectories).mean(axis=0)
# averageMove = averageMove * dataset.std_traj + dataset.mean_traj
df_sequence = pd.DataFrame(averageMove, columns=['dx','dy'])
df_sequence['velocity'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2) / dataset.FIXED_TIMESTEP
df_target = pd.DataFrame(np.array(dataset.buttonTargets).mean(axis=0), columns=['width','height','start_x','start_y'])
sequence_id = 0
dataset.SHOW_ONE = True
dataset.SHOW_ALL = False
df_abs = dataset.convertToAbsolute(df_sequence, df_target)
fig = dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id)

In [ ]:
# df_cleanedSeq, buttonTarget = dataset.processMouseData(SHOW_ALL=False)
# df_abs = dataset.convertToAbsolute(df_cleanedSeq, buttonTarget)
# dataset.plotTrajectory(df_abs, buttonTarget, 0)
# dataloader = getDataloader(norm_input_trajectories, norm_buttonTargets, BATCH_SIZE)

# df_sequence, df_target, start_x, start_y,left, top = dataset.processMouseData(SHOW_ONE=True, num_sequences=0)
# sequence_id = 0
# dataset.SHOW_ONE = True
# df_abs = dataset.convertToAbsolute(df_sequence, df_target)
# dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id)

In [ ]:
BATCH_SIZE = 256
dataloader = getDataloader(norm_input_trajectories, norm_buttonTargets, BATCH_SIZE)

### checking the dataloader

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for i, data in enumerate(dataloader, 0): 
    _input_trajectories_padded, _buttonTargets, trajectoryLengths = data
    # print(_input_trajectories_padded[0])
    if i == 3:
        break
    for ii in range(len(_input_trajectories_padded)):
        df_sequence = pd.DataFrame(_input_trajectories_padded[ii] * dataset.std_traj + dataset.mean_traj, columns=['dx','dy'])
        df_sequence['velocity'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2) / dataset.FIXED_TIMESTEP
        df_target = pd.DataFrame(_buttonTargets[ii] * dataset.std_button + dataset.mean_button, columns=['width','height','start_x','start_y'])
        sequence_id = 0
        dataset.SHOW_ONE = True
        df_abs = dataset.convertToAbsolute(df_sequence, df_target)

        fig.add_trace(go.Scatter(x=df_abs['x'], y=df_abs['y'],
                mode='lines+markers',
                marker=dict(
                            size=5, 
                            # symbol= "arrow-bar-up", angleref="previous",
                            # size=15,
                            # color='grey',),
                            color=df_abs['velocity'], colorscale='Viridis', showscale=True, colorbar=dict(title="Velocity")),
                
                ))
fig.update_layout(
    width=800,
    height=800,)
# fig.show()
pass

In [ ]:
from src.mouseGAN.models import WGAN_GP
LOAD_PRETRAINED = True

num_epochs = 20
num_feats = norm_input_trajectories[0].shape[1]
MAX_GRAD_NORM = 1000
latent_dim = 100
num_target_feats = norm_buttonTargets[0].shape[1]
MAX_SEQ_LEN = norm_input_trajectories[0].shape[0]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan = WGAN_GP(device, num_feats, num_target_feats, MAX_SEQ_LEN, miniBatchDisc=True, latent_dim=latent_dim)
if LOAD_PRETRAINED:
    gan.loadPretrained()

gan.train(dataloader, num_epochs, modelSaveInterval=3)

In [ ]:
gan.eval()
# z = torch.empty([1, MAX_SEQ_LEN, num_feats]).uniform_().to(device) # random vector
# sampling from spherical distribution
meanG = []
import plotly.graph_objects as go
fig = go.Figure()
for i in range(10):
    for x in range(-100,100, 10):
        z = torch.randn([1, MAX_SEQ_LEN, num_feats]).to(device)
        z = z / z.norm(dim=-1, keepdim=True)

        rawInput = np.array([149.59375,    100.0,       x,      100])
        norm_rawInput = (rawInput - dataset.mean_button) / dataset.std_button
        buttonTarget = torch.tensor([norm_rawInput], dtype=torch.float32).to(device)

        g_states = gan.generator.init_hidden(1)
        d_state = gan.discriminator.init_hidden(1)

        # feed inputs to generator
        g_feats, _ = gan.generator(z, buttonTarget, g_states)
        g_feats = g_feats.squeeze(0)
        # meanG.append(g_feats.mean(dim=0).cpu().detach().numpy())

        # convert back 
        g_feats = g_feats.cpu().detach().numpy()

        input_trajectories, buttonTargets = dataset.denormalize([g_feats], [norm_rawInput])
        input_trajectory = input_trajectories[0]
        buttonTarget = buttonTargets[0]
        df_sequence = pd.DataFrame(input_trajectory, columns=dataset.trajColumns)
        df_target = pd.DataFrame([rawInput], columns=dataset.targetColumns)
        sequence_id = 0
        print("starting location ", rawInput[-2:])
        dataset.SHOW_ONE = True
        # display(df_sequence)
        # display(df_target)
        start_x = rawInput[-2]
        start_y = rawInput[-1]
        sequence_id = 0
        dataset.SHOW_ONE = True

        df_sequence['distance'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2)
        df_sequence['velocity'] = df_sequence['distance'] / dataset.FIXED_TIMESTEP
        df_abs = dataset.convertToAbsolute(df_sequence, df_target)
        dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id, fig=fig)

In [1]:

def plotGeneratorSamples():
  fig = go.Figure()
  AXIAL_RESOLUTION = 10
  theta = np.linspace(0, 2*np.pi, AXIAL_RESOLUTION)
  low_radius = 100
  high_radius = 1000
  TOTAL_SAMPLES = 10

  trajectories = []
  buttonTargets = []
  maxRadius = 0

  TARGET_WIDTH = 150
  TARGET_HEIGHT = 100

  for i in range(TOTAL_SAMPLES // AXIAL_RESOLUTION):
      radius = np.random.random() * (high_radius - low_radius) + low_radius
      radius = 200
      maxRadius = max(maxRadius, radius)
      x = radius * np.cos(theta) 
      y = radius * np.sin(theta)
      for (x1,y1) in zip(x,y):
          z = torch.randn([1, MAX_SEQ_LEN, num_feats]).to(device)
          z = z / z.norm(dim=-1, keepdim=True)

          rawInput = np.array([TARGET_WIDTH,    TARGET_HEIGHT,       x1,      y1])

          norm_rawInput = (rawInput - dataset.mean_button) / dataset.std_button
          buttonTarget = torch.tensor([norm_rawInput], dtype=torch.float32).to(device)

          g_states = gan.generator.init_hidden(1)
          d_state = gan.discriminator.init_hidden(1)

          # feed inputs to generator
          g_feats, _ = gan.generator(z, buttonTarget, g_states)
          g_feats = g_feats.squeeze(0)
          # meanG.append(g_feats.mean(dim=0).cpu().detach().numpy())

          # convert back 
          g_feats = g_feats.cpu().detach().numpy()

          input_trajectories, buttonTargets = dataset.denormalize([g_feats], [norm_rawInput])
          input_trajectory = input_trajectories[0]
          buttonTarget = buttonTargets[0]
          df_sequence = pd.DataFrame(input_trajectory, columns=dataset.trajColumns)
          df_target = pd.DataFrame([rawInput], columns=dataset.targetColumns)
          sequence_id = 0
          # print("starting location ", rawInput[-2:])
          dataset.SHOW_ONE = True
          # display(df_sequence)
          # display(df_target)
          start_x = rawInput[-2]
          start_y = rawInput[-1]
          sequence_id = 0
          dataset.SHOW_ONE = True

          df_sequence['distance'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2)
          df_sequence['velocity'] = df_sequence['distance'] / dataset.FIXED_TIMESTEP
          df_abs = dataset.convertToAbsolute(df_sequence, df_target)

          sequence_id = 0
          dataset.SHOW_ONE = True
          fig.add_trace(go.Scatter(x=df_abs['x'], y=df_abs['y'],
                  mode='lines+markers',
                  marker=dict(
                              size=5, 
                              # symbol= "arrow-bar-up", angleref="previous",
                              # size=15,
                              # color='grey',),
                              color=df_abs['velocity'], colorscale='Viridis', showscale=True, colorbar=dict(title="Velocity")),
                  
                  ))
  x0, y0 = -TARGET_WIDTH/2, -TARGET_HEIGHT/2
  x1, y1 =  TARGET_WIDTH/2, TARGET_HEIGHT/2
  square = go.layout.Shape(
      type='rect',
      x0=x0,
      y0=y0,
      x1=x1,
      y1=y1,
      line=dict(color='black', width=2),
      fillcolor='rgba(0, 0, 255, 0.3)',
  )

  fig.update_layout(
      shapes=[square],
      width=800,
      height=800,
      xaxis=dict(
          range=[-maxRadius*1.1, maxRadius*1.1],)
      ,yaxis=dict(
          range=[-maxRadius*1.1, maxRadius*1.1],)
  )
  fig.show()

plotGeneratorSamples()

# for epoch in [10,20,30,40,50]:
#     latest_g_model = find_epoch_model('g', epoch, CKPT_DIR)
#     latest_d_model = find_epoch_model('d', epoch, CKPT_DIR)
#     if latest_g_model is not None:
#         model['g'].load_state_dict(torch.load(latest_g_model))
#         print(f"Loaded generator model: {latest_g_model}")
#     if latest_d_model is not None:
#         model['d'].load_state_dict(torch.load(latest_d_model))
#         print(f"Loaded discriminator model: {latest_d_model}")
#     epoch = min(int(latest_g_model.split('/')[-1].split('.')[0][1:]), int(latest_d_model.split('/')[-1].split('.')[0][1:]))
#     print(f"Starting from epoch {epoch}")
#     plotGeneratorSamples()


NameError: name 'go' is not defined